# Azure AutoML Project: Amazon Product Metadata Analysis
**Author:** [Your Name]  
**Date:** 2025-05-11

This notebook demonstrates Azure AutoML steps to train and evaluate a model predicting product ratings based on metadata from the Amazon dataset.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from azureml.core import Workspace, Dataset, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails

ModuleNotFoundError: No module named 'azureml'

In [ ]:
# Connect to Azure ML workspace
ws = Workspace.from_config()

In [ ]:
# Load dataset from local file or datastore (already uploaded via Data Factory)
# Example: AmazonProducts.csv uploaded to datastore and registered as Dataset
dataset = Dataset.get_by_name(ws, name='AmazonProducts')
df = dataset.to_pandas_dataframe()
df.head()

In [ ]:
# Clean and prepare the data
df = df.dropna(subset=['price', 'review_count', 'average_rating'])
df['review_count'] = df['review_count'].astype(int)
df['price'] = df['price'].astype(float)
df = df[['price', 'review_count', 'average_rating']]
df.describe()

In [ ]:
# Define AutoML configuration
automl_config = AutoMLConfig(
    task='regression',
    primary_metric='r2_score',
    training_data=dataset,
    label_column_name='average_rating',
    n_cross_validations=5,
    enable_early_stopping=True,
    experiment_timeout_minutes=30,
    compute_target='cpu-cluster'
)

In [ ]:
# Run AutoML experiment
experiment = Experiment(ws, 'amazon-products-automl')
run = experiment.submit(automl_config, show_output=True)

In [ ]:
# Show leaderboard
best_run, fitted_model = run.get_output()
print(fitted_model)
RunDetails(run).show()